## 2 кейс

**Соберите данные из Google Sheets и напишите функцию `generate_report`, которая принимает три списка (данные с каждого листа в Google Sheets) и сохраняет отчет в файл `student_debt_report.txt`**

**Важно**

Перед началом решения задачи выполните следующую ячейку - в ней скачиваются нужные файлы

In [1]:
!wget https://gist.github.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json

!wget https://gist.github.com/Vs8th/39c5deed0f5539d781f00328f7fd4fe0/raw/result.txt

--2024-10-23 14:57:58--  https://gist.github.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json
Resolving gist.github.com (gist.github.com)... 140.82.112.3
Connecting to gist.github.com (gist.github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json [following]
--2024-10-23 14:57:58--  https://gist.githubusercontent.com/Vs8th/d0bd4bdbbb58c8ae4f70a2a503e2d5fc/raw/creds.json
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2358 (2.3K) [text/plain]
Saving to: ‘creds.json’

creds.json          100%[===================>]   2.30K  --.-KB/s    in 0s      

2024-10-23 14:57:58 (36.0 MB/s) - ‘creds.json’ saved [2

Чтобы посмотреть как выглядят сами гугл таблицы в оригинале - можете перейти по [ссылке](https://docs.google.com/spreadsheets/d/1hRnw-PEftF0J-6KY7InlILVwWdkJu4vJiGwRjuE_P4U/edit?usp=sharing) и изучить.

Как примерно должен выглядеть результат:

In [2]:
with open('result.txt') as f:
  print(f.read())

Студент Иванов У.У. - долг 100000 рублей
Студент Петров Е.Е. - долг 250000 рублей
Студент Сидоров И.И. - долг 10000 рублей


In [ ]:
#@title Если возникнут трудности с определением `scope`, подсказка:

scope = ['https://www.googleapis.com/auth/spreadsheets.readonly',
         'https://www.googleapis.com/auth/drive']

**Примечание**

Считать должников необходимо на 1 марта 2023 года. То есть определяя количество просроченных платежей, мы определяем их количество не на настоящую дату, а на **1 марта 2023 года**. А периоды внесения платежей для всех студентов одинаковы - **6 месяцев**.

То есть, если ожидаемый платеж должен был пройти 3 февраля 2022 года, то к 1 марта 2023 студент просрочил уже три платежа (3 февраля 2022, 3 августа 2022 и 3 февраля 2023).

In [3]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [4]:
import pandas as pd

**Решение**

Напишите свое решение ниже

In [74]:
def generate_report(sheet1, sheet2, sheet3):
  sheet = client.open("Installments").worksheet(sheet1)
  sheet1_data = sheet.get_all_records()

  sheet = client.open("Installments").worksheet(sheet2)
  sheet2_data = sheet.get_all_records()

  sheet = client.open("Installments").worksheet(sheet3)
  sheet3_data = sheet.get_all_records()

  sheet1_df = pd.DataFrame(sheet1_data)
  sheet2_df = pd.DataFrame(sheet2_data)
  sheet3_df = pd.DataFrame(sheet3_data)
  sh_1_2_df = sheet1_df.join(sheet2_df.set_index('student_id'), on = 'student_id', how = 'inner')
  all_data = sh_1_2_df.join(sheet3_df.set_index('student_id'), on = 'student_id', how = 'inner')

  all_data['last_payment_date'] =  pd.to_datetime(all_data['last_payment_date'], format='%d.%m.%Y')
  all_data['expected_payment_date'] =  pd.to_datetime(all_data['expected_payment_date'], format='%d.%m.%Y')

  target_date = pd.to_datetime('2023-03-01')
  all_data = all_data[all_data['expected_payment_date'] <= target_date]

  all_data['delta_days'] = target_date - all_data['last_payment_date']

  all_data['delta'] = all_data['delta_days'].dt.days / 182 # 6 мес
  all_data['dolg'] = all_data['one-time_payment'] * all_data['delta'].astype(int)

  lines = []
  for idx, row in all_data[['student_name', 'left_to_pay', 'dolg']].iterrows():
    if row['dolg'] > row['left_to_pay']:
      lines.append(f'Студент {row["student_name"]} - долг {row["left_to_pay"]} рублей')
    else:
      lines.append(f'Студент {row["student_name"]} - долг {row["dolg"]} рублей')

  with open('student_debt_report.txt', 'w') as f:
    f.write('\n'.join(lines))

In [75]:
generate_report('Лист1', 'Лист2', 'Лист3')

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [91]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt

import pandas as pd

user_answer = pd.read_csv('student_debt_report.txt')
correct_answer = pd.read_csv('student_debt.txt')

--2024-10-23 15:54:20--  https://gist.github.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt
Resolving gist.github.com (gist.github.com)... 140.82.113.3
Connecting to gist.github.com (gist.github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt [following]
--2024-10-23 15:54:20--  https://gist.githubusercontent.com/Vs8th/63832f093f4db8d8b251ba5d39571f3d/raw/student_debt.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11325 (11K) [text/plain]
Saving to: ‘student_debt.txt.1’

student_debt.txt.1  100%[===================>]  11.06K  --.-KB/s    in 0s      

2024-10-23 15:54:21 (97.5 MB/

In [92]:
correct_answer

,Студент Кузнецова К.А. - долг 266666 рублей
0,Студент Петров К.А. - долг 66666 рублей
1,Студент Иванов А.А. - долг 100000 рублей
2,Студент Иванов П.П. - долг 116666 рублей
3,Студент Петров И.И. - долг 116666 рублей
4,Студент Петров А.А. - долг 16666 рублей
...,...
160,Студент Сидоров А.А. - долг 266666 рублей
161,Студент Иванов М.А. - долг 25000 рублей
162,Студент Иванов М.А. - долг 387227 рублей
163,Студент Николаева М.А. - долг 33333 рублей


In [93]:
user_answer

,Студент Кузнецова К.А. - долг 266666 рублей
0,Студент Петров К.А. - долг 66666 рублей
1,Студент Иванов А.А. - долг 100000 рублей
2,Студент Иванов П.П. - долг 116666 рублей
3,Студент Петров И.И. - долг 116666 рублей
4,Студент Петров А.А. - долг 16666 рублей
...,...
160,Студент Сидоров А.А. - долг 266666 рублей
161,Студент Иванов М.А. - долг 25000 рублей
162,Студент Иванов М.А. - долг 387227 рублей
163,Студент Николаева М.А. - долг 33333 рублей


In [94]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
  assert user_answer.columns.equals(correct_answer.columns), 'Названия столбцов не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!!')

Поздравляем, Вы справились и успешно прошли все проверки!!
